In [1]:
import numpy as np
import pandas
import time
import elpigraph
import matplotlib.pyplot as plt
from copy import deepcopy
import rpy2.robjects.packages as rpackages
import rpy2.robjects
import rpy2.robjects.numpy2ri
import rpy2.robjects.pandas2ri
r_elpigraph = rpackages.importr("ElPiGraph.R")
rpy2.robjects.numpy2ri.activate()
rpy2.robjects.pandas2ri.activate()
plt.style.use('seaborn')
np.random.seed(0)

ModuleNotFoundError: No module named 'scipy.spatial.qhull._qhull'; 'scipy.spatial.qhull' is not a package

In [2]:
# Load the example data or real data
data =  np.genfromtxt('data/tree_data.csv', delimiter=',')

# Create desired list of inputs for R and Python
input_data = [data]*5
epg_n_nodes = [20,25,15,20,30]
epg_lambda = [.1,.02,.7,.03,.08]
epg_mu = [.02,.07,.01,.04,.06]
epg_trimmingradius = [float('inf'),.7,.8,.6,.5]
epg_finalenergy = ['Penalized','Base','Penalized','Base','Base']
epg_alpha = [.01,.03,.05,.08,.04]
epg_beta = [.03,.02,.04,.07,.01]
epg_mode = [2,1,1,2,1]
epg_n_processes = [1,2,1,2,1]
epg_collapse_mode = ['PointNumber','PointNumber_Extrema','PointNumber_Leaves','EdgesNumber','EdgesLength']
epg_collapse_par = [5,7,4,6,3]
epg_maxsteps = [float('inf'),1000,100,20,200]
                                  # Python uses WeightedCentroid not Weigthed (corrected typo)
epg_ext_mode =   ['QuantDists','QuantCentroid','WeightedCentroid','QuantCentroid','WeightedCentroid']
r_epg_ext_mode = ['QuantDists','QuantCentroid','WeigthedCentroid','QuantCentroid','WeigthedCentroid'] 
epg_ext_par = [.5,.6,.8,.9,.5]
epg_shift_mode = ['NodeDensity','NodePoints','NodeDensity','NodePoints','NodeDensity']
epg_shift_radius = [0.05,0.07,0.04,0.08,0.03]
epg_shift_max = [5,7,4,8,6]



# Results storage Python
epg_main = []
epg_obj_collapse = []
epg_obj_shift = []
epg_obj_extend = []
epg_obj_fineTune = []
# Results storage R
r_epg_main = []
r_epg_obj_collapse = []
r_epg_obj_shift = []
r_epg_obj_extend = []
r_epg_obj_fineTune = []

for i in range(len(input_data)):
    
    ############################ Run functions, Python version ###################################
    
    epg_main.append(elpigraph.computeElasticPrincipalTree(X=input_data[i],NumNodes = epg_n_nodes[i], 
                                                          Lambda=epg_lambda[i], Mu=epg_mu[i],
                                                          TrimmingRadius = epg_trimmingradius[i],
                                                          FinalEnergy = epg_finalenergy[i],
                                                          alpha = epg_alpha[i],
                                                          beta = epg_beta[i],                                                    
                                                          Do_PCA=False,CenterData=False,
                                                          n_cores = epg_n_processes[i],
                                                          nReps=1,
                                                          EmbPointProb=1.0,
                                                          Mode = epg_mode[i],
                                                          MaxSteps = epg_maxsteps[i])[0])
    
    # util functions input
    epg_obj = epg_main[i]
    init_nodes_pos = epg_obj['NodePositions']
    init_edges = epg_obj['Edges'][0]
    #########################################
    epg_obj_collapse.append(elpigraph.CollapseBranches(X = input_data[i], PG = epg_obj, Mode = epg_collapse_mode[i], ControlPar = epg_collapse_par[i]))


    epg_obj_shift.append(elpigraph.ShiftBranching(X = input_data[i], 
                                                  PG = epg_obj, 
                                                  TrimmingRadius = epg_trimmingradius[i],                       
                                                  SelectionMode = epg_shift_mode[i], 
                                                  DensityRadius = epg_shift_radius[i],
                                                  MaxShift = epg_shift_max[i]))
    
    epg_obj_extend.append(elpigraph.ExtendLeaves(X = input_data[i], 
                                                 PG = epg_obj,
                                                 TrimmingRadius = epg_trimmingradius[i],
                                                 Mode = epg_ext_mode[i], 
                                                 ControlPar = epg_ext_par[i],
                                                 DoSA_maxiter=4000)) #number of iterations for simulated annealing
    
    epg_obj_fineTune.append(elpigraph.fineTuneBR(X=input_data[i],
                                                MaxSteps = epg_maxsteps[i],
                                                Mode = 2,
                                                NumNodes = epg_n_nodes[i], 
                                                InitNodePositions = init_nodes_pos,
                                                InitEdges=init_edges,
                                                Lambda=epg_lambda[i], Mu=epg_mu[i],
                                                TrimmingRadius= epg_trimmingradius[i],
                                                FinalEnergy = epg_finalenergy[i],
                                                alpha = epg_alpha[i],
                                                beta = epg_beta[i],                                                    
                                                Do_PCA=False,CenterData=False,
                                                n_cores = epg_n_processes[i],
                                                nReps=1,
                                                ProbPoint=1.0)[0])
    
    ############################ Run functions, R version ###################################

    tmp = r_elpigraph.computeElasticPrincipalTree(X=input_data[i],NumNodes = epg_n_nodes[i], 
                                                  Lambda=epg_lambda[i], Mu=epg_mu[i],
                                                  TrimmingRadius= epg_trimmingradius[i],
                                                  FinalEnergy = epg_finalenergy[i],
                                                  alpha = epg_alpha[i],
                                                  beta = epg_beta[i],                                                    
                                                  Do_PCA=False,CenterData=False,
                                                  n_cores = epg_n_processes[i],
                                                  nReps=1,
                                                  ProbPoint=1.0,
                                                  drawPCAView=False,
                                                  Mode = epg_mode[i],
                                                  MaxSteps = epg_maxsteps[i])[0]
    
    r_epg_main.append(dict(zip(tmp.names, map(list,np.array(tmp))))) # Convert R result to dict format
    
    # util functions input
    r_epg_obj = tmp
    init_nodes_pos = r_epg_obj[0]
    init_edges = r_epg_obj[1][0]
    #########################################
    try:
        r_epg_obj_collapse.append(r_elpigraph.CollapseBrances(X = input_data[i], TargetPG = r_epg_obj, Mode = epg_collapse_mode[i], ControlPar = epg_collapse_par[i]))
    except:
        r_epg_obj_collapse.append('bug')

    r_epg_obj_shift.append(r_elpigraph.ShiftBranching(X = input_data[i], 
                                                      TargetPG = r_epg_obj, 
                                                      TrimmingRadius = epg_trimmingradius[i],                       
                                                      SelectionMode = epg_shift_mode[i], 
                                                      DensityRadius = epg_shift_radius[i],
                                                      MaxShift = epg_shift_max[i]))
    
    tmp_ext = r_elpigraph.ExtendLeaves(X = input_data[i], 
                                       TargetPG = r_epg_obj,
                                       TrimmingRadius = epg_trimmingradius[i],
                                       Mode = r_epg_ext_mode[i], 
                                       ControlPar = epg_ext_par[i],
                                       PlotSelected = False)
    r_epg_obj_extend.append(dict(zip(tmp_ext.names, map(list,np.array(tmp_ext)))))
    
    tmp_fineTune = r_elpigraph.fineTuneBR(X=input_data[i],
                                          MaxSteps = epg_maxsteps[i],
                                          Mode = 2,
                                          NumNodes = epg_n_nodes[i], 
                                          InitNodePositions = init_nodes_pos,
                                          InitEdges=init_edges,
                                          Lambda=epg_lambda[i], Mu=epg_mu[i],
                                          TrimmingRadius= epg_trimmingradius[i],
                                          FinalEnergy = epg_finalenergy[i],
                                          alpha = epg_alpha[i],
                                          beta = epg_beta[i],                                                    
                                          Do_PCA=False,CenterData=False,
                                          drawAccuracyComplexity = False, drawEnergy = False,drawPCAView = False,
                                          n_cores = epg_n_processes[i],
                                          nReps=1,
                                          ProbPoint=1.0)[0]
    r_epg_obj_fineTune.append(dict(zip(tmp_fineTune.names, map(list,np.array(tmp_fineTune)))))

guration will be ignored
Computing EPG with  20  nodes on  492  points and  3  dimensions
Nodes =  23 4 56 7 89 1011 12 1314 15 16171819

BARCODE	ENERGY	NNODES	NEDGES	NRIBS	NSTARS	NRAYS	NRAYS2	MSE	MSEP	FVE	FVEP	UE	UR	URN	URN2	URSD

2||20	0.0841	20	19	14	2	0	0	0.0357	0.0343	0.9337	0.9363	0.0475	0.0008	0.0165	0.3296	0


3.5791  seconds elapsed
Removing the terminal branch with nodes: [14 17]
Moving the branching point at node 0
Moving the branching point at node 14
Performing simulated annealing. This may take a while
Performing simulated annealing. This may take a while
Performing simulated annealing. This may take a while
Performing simulated annealing. This may take a while
(4, 3)
(20, 3)
Constructing tree 1 of 1 / Subset 1 of 1
Computing EPG with  20  nodes on  492  points and  3  dimensions
Using grammar optimization
When setting GrammarOptimization to TRUE, MaxSteps must be finite. Using MaxSteps = 1
Nodes =  20 20 

BARCODE	ENERGY	NNODES	NEDGES	NRIBS	NSTARS	NRAYS	NRAYS2	MSE	MSEP	F

### Step 2 : check final output : computePrincipalTree, ExtendLeaves, fineTuneBR
#### For dict keys in (NodePositions, Edges, FinalReport, ElasticMatrix) -> prints key, iteration index, function if a difference is found in the result dictionary

In [3]:
funcs = ['computeElasticPrincipalTree','ExtendLeaves','fineTuneBR']
j = 0  #funcs index
for res_py,res_R in [(epg_main,r_epg_main),(epg_obj_extend,r_epg_obj_extend),(epg_obj_fineTune,r_epg_obj_fineTune)]: # for func in funcs
    for i in range(len(input_data)): # check each set of output
        one_res_py = res_py[i]
        one_res_R = res_R[i]
        
        for key in one_res_py:
            if key == 'NodePositions':
                try: assert np.allclose(one_res_py[key], one_res_R[key])
                except: print(key,i,funcs[j])

            if key == 'Edges':
                try: assert all(map(lambda x:np.all(x),[one_res_py[key][0]==(one_res_R[key][0]-1), #correcting R indexing that starts at one
                                                        one_res_py[key][1][~np.isnan(one_res_py[key][1])]==one_res_R[key][1][~np.isnan(one_res_R[key][1])]]))
                                                        #one_res_py[key][2][~np.isnan(one_res_py[key][2])]==one_res_R[key][2][~np.isnan(one_res_R[key][2])]]))
                except: print(key,i,funcs[j])

            if key == 'FinalReport':
                try: assert(np.allclose(np.array(list(one_res_py[key].values()))[1:].astype(float), 
                                        np.array(one_res_R[key]).flatten()[1:].astype(float)))
                except: print(key,i, funcs[j])

            if key == 'ElasticMatrix':
                try: assert np.all(one_res_py[key] == one_res_R[key])
                except: print(key,i,funcs[j])
    j+=1

NodePositions 0 ExtendLeaves


In [4]:
### This failure case for ExtendLeaves happens when using the 'QuantDists' option (annealing procedure). We can see the added nodes are still quite close, though
print(epg_obj_extend[0]['NodePositions'][-4:])
print('\n',np.array(r_epg_obj_extend[0]['NodePositions'][-4:]))

[[-1.33051535  0.88428135  0.04640507]
 [ 1.21694761  0.94293646  0.00552228]
 [ 0.13717873  1.18861498  0.00747918]
 [-0.03053228 -0.99463781  0.02592915]]

 [[-1.32606864  0.90010366  0.01065414]
 [ 1.2215765   0.9373269   0.0347547 ]
 [ 0.07501994  1.12699983 -0.00160104]
 [-0.0130761  -0.993484    0.03831469]]


### Step 3 : check CollapseBranches, ShiftBranching

In [6]:
for i in range(len(input_data)):
    
    # CollapseBranches         
    try: 
        r_collapse_dict = dict(zip(r_epg_obj_collapse[i].names,r_epg_obj_collapse[i])) 
        # Edges might be correct but differ in ordering between Python and R version. We double-check that
        test_rows = []
        for row in epg_obj_collapse[i]['Edges']:
            test_rows.append(any([all(row==row_r) for row_r in (r_collapse_dict['Edges']-1)]))
        assert (all(test_rows) and np.allclose(epg_obj_collapse[i]['Nodes'],r_collapse_dict['Nodes']))

    except: 
        #Collapse branches can bug for input parameters, or return empty array. We check one of these cases happened for both versions
        try: 
            for obj in [epg_obj_collapse[i],r_epg_obj_collapse[i]]:
                if type(obj) ==str:
                    assert obj == 'bug'  #check if function bugged
                elif type(obj) ==  dict: 
                    assert np.array(obj['Edges']).size == 0    #or check if array is empty, Python
                elif hasattr(np.array(obj[0]),'size'):
                    assert np.array(obj[0]).size == 0          #or check if array is empty, R
            print('CollapseBranches failed for both versions',i)
        except: print('CollapseBranches', i)
        

    # ShiftBranching
    try: 
        # Edges might be correct but differ in ordering. We double-check that :
        r_shift_dict = dict(zip(r_epg_obj_shift[i].names,r_epg_obj_shift[i]))          
        sort_shift_py = epg_obj_shift[i]['Edges'][:,1].argsort()
        sort_shift_r = (r_shift_dict['Edges']-1)[:,1].argsort()
        assert all([np.all(np.array(epg_obj_shift[i]['Edges'][sort_shift_py])==(r_shift_dict['Edges']-1)[sort_shift_r]),  #correcting R indexing that starts at one
                     np.allclose(epg_obj_shift[i]['NodePositions'],r_shift_dict['NodePositions'])])
    except: print('ShiftBranching',i)